Ethan Buege <br> ISA 414 Final Project


In [78]:
# standard python libraries
import requests
from requests import auth
from time import sleep
import datetime as dt
from datetime import timezone
import collections
import random
import logging

# data handling
import pandas as pd
import numpy as np
import re
import pickle
import pymongo

# reddit API stuff
import praw
from psaw import PushshiftAPI
from praw.models import MoreComments

# stuff for doc2vec encoding
import gensim
from sklearn.model_selection import train_test_split

# pytorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchtext as tt


logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

Initialize API connections

In [3]:
user_agent = 'sentiment-analysis'
client_id = 'YPYBeLM43W6kliCI4HEvlQ'
client_secret = 'gqFB1AnxPgMiRnQHWEXZ2xl6n1QliQ'

reddit_auth = requests.auth.HTTPBasicAuth(client_id, client_secret)
headers = {'User-Agent': 'sentiment-analysis'}

# res = requests.post(url='https://www.reddit.com/api/v1/access_token',
#                     auth=auth, data=data, headers=headers)
# token = res.json()['access_token']
# headers = {**headers, **{'Authorization': f"bearer {token}"}}

reddit = praw.Reddit(client_id=client_id, client_secret=client_secret, user_agent=user_agent)
ps_api = PushshiftAPI(reddit)
wsb = reddit.subreddit("wallstreetbets")

# print(reddit.user.me())

client = pymongo.MongoClient("mongodb://admin:abcd1234@"
                             "cluster0-shard-00-00.3y0ug.mongodb.net:27017,"
                             "cluster0-shard-00-01.3y0ug.mongodb.net:27017,"
                             "cluster0-shard-00-02.3y0ug.mongodb.net:27017"
                             "/ISA414?ssl=true&replicaSet=atlas-j9cpkb-shard-0&"
                             "authSource=admin&retryWrites=true&w=majority")
db = client.Reddit
collection = db.WallStreetBets

# for submission in wsb.stream.submissions():
#     print(submission.title)

Get training data by scraping the WSB subreddit
Group posts by stock symbol (ticker) and week
DO NOT RUN UNLESS ABSOLUTELY NECESSARY (takes 12+ hours)

In [102]:
# one_week_in_seconds = 604800
# start_epoch = int(dt.datetime(2022, 4, 20).timestamp())
# subreddit = 'wallstreetbets'
# ticker = 'TSLA'
# tsla_submissions = list()


def get_historical_posts(subreddit, ticker, years):
    weekly_submissions = list()
    api_filter = ['id', 'subreddit', 'link_flair_text', 'title', 'selftext', 'score', 'upvote_ratio']

    one_week_in_seconds = 604800

    now = dt.datetime.now(timezone.utc)
    utc_time = now.replace(tzinfo=timezone.utc)
    start_epoch = int(dt.datetime(2022, 4, 18).timestamp())
    end_epoch = start_epoch - one_week_in_seconds
    for year in range(years):
        for week in range(52):
            gen = ps_api.search_submissions(before=start_epoch, after=end_epoch, subreddit=subreddit,
                                            filter=api_filter, title=ticker)
            results = [s for s in gen]

            weekly_submissions.append(results)

            start_epoch = end_epoch
            end_epoch -= one_week_in_seconds

            print(f'Fetched {len(results)} new {ticker} posts from {(week + 1) + (52 * year)} weeks ago')
        print(f'Cumulative {ticker} submissions: {sum([len(w) for w in weekly_submissions])}')

    return weekly_submissions


top_10_tickers = ['GME', 'AMC', 'TSLA', 'AAPL', 'AMZN', 'AMD', 'NVDA', 'MSFT', 'SPY', 'QQQ']

for ticker in top_10_tickers:
    sub_history = get_historical_posts('wallstreetbets', ticker, 5)
    pickle.dump(sub_history, open(f'E:/reddit_data/{ticker}.p', "wb"))


# print(len(tsla_submissions))
# pickle.dump(tsla_submissions, open(f'E:/reddit_data/{ticker}.p', "wb"))


Fetched 49 new GME posts from 1 weeks ago
Cumulative GME submissions: 49
Fetched 100 new GME posts from 2 weeks ago
Fetched 481 new GME posts from 3 weeks ago
Fetched 479 new GME posts from 4 weeks ago
Fetched 86 new GME posts from 5 weeks ago
Fetched 41 new GME posts from 6 weeks ago
Fetched 51 new GME posts from 7 weeks ago
Fetched 48 new GME posts from 8 weeks ago
Fetched 63 new GME posts from 9 weeks ago


C:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\psaw\PushshiftAPI.py:192: UserWarning: Got non 200 code 429
  warnings.warn("Got non 200 code %s" % response.status_code)
C:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\psaw\PushshiftAPI.py:180: UserWarning: Unable to connect to pushshift.io. Retrying after backoff.
  warnings.warn("Unable to connect to pushshift.io. Retrying after backoff.")


Fetched 93 new GME posts from 10 weeks ago
Fetched 83 new GME posts from 11 weeks ago
Cumulative GME submissions: 1574
Fetched 212 new GME posts from 12 weeks ago
Fetched 198 new GME posts from 13 weeks ago
Fetched 153 new GME posts from 14 weeks ago
Fetched 356 new GME posts from 15 weeks ago
Fetched 103 new GME posts from 16 weeks ago
Fetched 54 new GME posts from 17 weeks ago
Fetched 110 new GME posts from 18 weeks ago
Fetched 130 new GME posts from 19 weeks ago
Fetched 121 new GME posts from 20 weeks ago
Fetched 110 new GME posts from 21 weeks ago
Cumulative GME submissions: 3121
Fetched 115 new GME posts from 22 weeks ago
Fetched 139 new GME posts from 23 weeks ago
Fetched 231 new GME posts from 24 weeks ago
Fetched 103 new GME posts from 25 weeks ago
Fetched 111 new GME posts from 26 weeks ago
Fetched 86 new GME posts from 27 weeks ago
Fetched 120 new GME posts from 28 weeks ago
Fetched 89 new GME posts from 29 weeks ago
Fetched 87 new GME posts from 30 weeks ago
Fetched 124 new 

C:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\psaw\PushshiftAPI.py:192: UserWarning: Got non 200 code 502
  warnings.warn("Got non 200 code %s" % response.status_code)


Fetched 74952 new GME posts from 64 weeks ago
Fetched 6063 new GME posts from 65 weeks ago
Fetched 3199 new GME posts from 66 weeks ago
Fetched 401 new GME posts from 67 weeks ago
Fetched 561 new GME posts from 68 weeks ago
Fetched 468 new GME posts from 69 weeks ago
Fetched 152 new GME posts from 70 weeks ago
Fetched 287 new GME posts from 71 weeks ago
Fetched 504 new GME posts from 72 weeks ago
Fetched 266 new GME posts from 73 weeks ago
Cumulative GME submissions: 163392
Fetched 54 new GME posts from 74 weeks ago
Fetched 37 new GME posts from 75 weeks ago


C:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\psaw\PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)


Fetched 13 new GME posts from 76 weeks ago
Fetched 75 new GME posts from 77 weeks ago
Fetched 43 new GME posts from 78 weeks ago
Fetched 71 new GME posts from 79 weeks ago
Fetched 94 new GME posts from 80 weeks ago
Fetched 6 new GME posts from 81 weeks ago
Fetched 22 new GME posts from 82 weeks ago
Fetched 21 new GME posts from 83 weeks ago
Cumulative GME submissions: 163828
Fetched 14 new GME posts from 84 weeks ago
Fetched 16 new GME posts from 85 weeks ago
Fetched 2 new GME posts from 86 weeks ago
Fetched 1 new GME posts from 87 weeks ago
Fetched 0 new GME posts from 88 weeks ago
Fetched 4 new GME posts from 89 weeks ago
Fetched 2 new GME posts from 90 weeks ago
Fetched 1 new GME posts from 91 weeks ago
Fetched 0 new GME posts from 92 weeks ago
Fetched 0 new GME posts from 93 weeks ago
Cumulative GME submissions: 163868
Fetched 1 new GME posts from 94 weeks ago
Fetched 0 new GME posts from 95 weeks ago
Fetched 2 new GME posts from 96 weeks ago
Fetched 11 new GME posts from 97 weeks 

In [50]:
top_10_tickers = ['GME', 'AMC', 'TSLA', 'AAPL', 'AMZN', 'AMD', 'NVDA', 'MSFT', 'SPY', 'QQQ']

for ticker in top_10_tickers:
    raw_data = pickle.load(open(f'E:/reddit_data/{ticker}.p', "rb"))

    total_submissions = sum([len(w) for w in raw_data])
    print(total_submissions)

    series_data = []
    for w in raw_data:
        series_data += w

    series_data.sort(key=lambda x: x.created_utc, reverse=False)

    pickle.dump(series_data, open(f'E:/reddit_data/{ticker}_series.p', "wb"))
# only_text_subs = []
# for week in tsla:
#     only_text_subs.append(list(filter(lambda x: x.selftext and x.selftext != '[removed]' and x.selftext != '[deleted]', week)))

# print(tsla[21][12].title)

164166
1650248864.0
1493993632.0
90112
1650248864.0
1498772038.0
13516
1650160498.0
1493051254.0
3151
1649970132.0
1493329852.0
2518
1649936184.0
1493071635.0
6627
1650014720.0
1493051054.0
2312
1650061548.0
1493025767.0
2278
1649831488.0
1493063001.0
10576
1650245649.0
1493038283.0
815
1649985120.0
1493319949.0


In [170]:
top_10_tickers = ['GME', 'AMC', 'TSLA', 'AAPL', 'AMZN', 'AMD', 'NVDA', 'MSFT', 'SPY', 'QQQ']
sym_regex = r'[$]?(GME|AMC|TSLA|AAPL|AMZN|AMD|NVDA|MSFT|SPY|QQQ)'
url_regex = r'(\w+://)?www.[\w\-\.]+'
all_posts = []
for ticker in top_10_tickers:
    raw_data = pickle.load(open(f'E:/reddit_data/{ticker}_series.p', "rb"))

    # replace stock symbols with the same token for all posts to help with word embeddings
    for sub in raw_data:
        sub.title = re.sub(sym_regex, 'sym', sub.title, flags=re.IGNORECASE)
        sub.title = re.sub(url_regex, 'link', sub.title)

        sub.selftext = re.sub(sym_regex, 'sym', sub.selftext, flags=re.IGNORECASE)
        sub.selftext = re.sub(url_regex, 'link', sub.selftext)

    pickle.dump(raw_data, open(f'E:/reddit_data/{ticker}_series.p', "wb"))

    all_posts += raw_data

# not efficient to sort after concatenating but doesn't really matter since its a one time operation
all_posts.sort(key=lambda x: x.created_utc, reverse=False)
print(len(all_posts))

# get title and text portion from all posts to train doc2vec model
all_text = []
for sub in all_posts:
    if 'deleted by user' not in sub.title and len(sub.title) > 5:
        if len(sub.selftext) > 20:
            all_text.append('\n'.join([sub.title, sub.selftext]))
        else:
            all_text.append(sub.title)

print(len(all_text))

pickle.dump(all_text, open(f'E:/reddit_data/raw_title_list.p', "wb"))

296071
283212


Preprocess text and create tagged documents.

In [237]:
# set seed so we can reproduce outcomes
np.random.seed(42069)

train_data, test_data = train_test_split(list(set(all_text)), test_size=0.2, train_size=0.8)

def tokenize_docs(doclist, tokens_only=False):
    for i, doc in enumerate(doclist):
        tokens = gensim.utils.simple_preprocess(doc)
        if tokens_only:
            yield tokens
        else:
            yield gensim.models.doc2vec.TaggedDocument(tokens, [i])


train_corpus = list(tokenize_docs(train_data))
test_corpus = list(tokenize_docs(test_data, True))

print(len(train_corpus))
print(len(test_corpus))

184118
46030


Define doc2vec model and build vocabulary

In [305]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=300, min_count=25, window=10, epochs=100)
model.build_vocab(train_corpus)

print(f"Word 'moon' appeared {model.wv.get_vecattr('moon', 'count')} times in the training corpus.")

2022-04-24 22:46:25,118 : INFO : Doc2Vec lifecycle event {'params': 'Doc2Vec(dm/m,d300,n5,w5,mc25,s0.001,t3)', 'datetime': '2022-04-24T22:46:25.118619', 'gensim': '4.1.2', 'python': '3.10.2 (tags/v3.10.2:a58ebcc, Jan 17 2022, 14:12:15) [MSC v.1929 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19044-SP0', 'event': 'created'}
2022-04-24 22:46:25,133 : INFO : collecting all words and their counts
2022-04-24 22:46:25,134 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2022-04-24 22:46:25,165 : INFO : PROGRESS: at example #10000, processed 117319 words (3796915/s), 8298 word types, 0 tags
2022-04-24 22:46:25,195 : INFO : PROGRESS: at example #20000, processed 232184 words (4027538/s), 11657 word types, 0 tags
2022-04-24 22:46:25,222 : INFO : PROGRESS: at example #30000, processed 347792 words (4332354/s), 14283 word types, 0 tags
2022-04-24 22:46:25,246 : INFO : PROGRESS: at example #40000, processed 465747 words (4920614/s), 16592 word types, 0 tags
2022-04

Word 'moon' appeared 11257 times in the training corpus.


Train model

In [306]:
model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)
print('done!')


2022-04-24 22:47:16,744 : INFO : Doc2Vec lifecycle event {'msg': 'training model with 3 workers on 3981 vocabulary and 300 features, using sg=0 hs=0 sample=0.001 negative=5 window=5 shrink_windows=True', 'datetime': '2022-04-24T22:47:16.744923', 'gensim': '4.1.2', 'python': '3.10.2 (tags/v3.10.2:a58ebcc, Jan 17 2022, 14:12:15) [MSC v.1929 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19044-SP0', 'event': 'train'}
2022-04-24 22:47:17,958 : INFO : EPOCH 1 - PROGRESS: at 4.62% examples, 63321 words/s, in_qsize 5, out_qsize 0
2022-04-24 22:47:19,050 : INFO : EPOCH 1 - PROGRESS: at 11.67% examples, 82438 words/s, in_qsize 6, out_qsize 0
2022-04-24 22:47:20,105 : INFO : EPOCH 1 - PROGRESS: at 18.66% examples, 90146 words/s, in_qsize 5, out_qsize 0
2022-04-24 22:47:21,175 : INFO : EPOCH 1 - PROGRESS: at 25.61% examples, 93759 words/s, in_qsize 5, out_qsize 0
2022-04-24 22:47:22,280 : INFO : EPOCH 1 - PROGRESS: at 32.98% examples, 96727 words/s, in_qsize 5, out_qsize 0
2022-04-24 22:47:23,385

done!


Model sanity check (see how often it identifies a document as most similar to itself) <br>
<i>note: sourced from doc2vec documentation</i>

In [307]:
ranks = []
pos = random.randint(0, len(train_corpus))
for doc_id in range(0, len(train_corpus), len(train_corpus) // 1000):
    inferred_vector = model.infer_vector(train_corpus[doc_id].words)
    sims = model.dv.most_similar([inferred_vector], topn=len(model.dv))
    rank = [docid for docid, sim in sims].index(doc_id)
    ranks.append(rank)

print(ranks)
counter1 = collections.Counter(ranks)
print(counter1)

Counter({0: 84, 4: 2, 19: 2, 1: 2, 7: 1, 1160: 1, 297: 1, 26: 1, 27: 1, 534: 1, 5: 1, 858: 1, 947: 1, 901: 1, 69608: 1})
Counter({(112125, 0.45198488235473633): 1, (124873, 0.39164525270462036): 1, (124385, 0.5829038023948669): 1, (83420, 0.538532555103302): 1, (159019, 0.4563300609588623): 1, (103838, 0.48993921279907227): 1, (181446, 0.5733926296234131): 1, (159045, 0.3333837687969208): 1, (160996, 0.6029463410377502): 1, (39132, 0.6868435144424438): 1, (166675, 0.44720813632011414): 1, (152849, 0.5109429955482483): 1, (36809, 0.6445577144622803): 1, (5489, 0.6661074757575989): 1, (144080, 0.46010375022888184): 1, (26469, 0.6365119218826294): 1, (56802, 0.36735859513282776): 1, (66553, 0.42701947689056396): 1, (111593, 0.7186262607574463): 1, (153464, 0.3855120539665222): 1, (55277, 0.623818039894104): 1, (105693, 0.45224595069885254): 1, (38338, 0.417990505695343): 1, (118055, 0.4883791506290436): 1, (26469, 0.3724164068698883): 1, (14913, 0.4831525981426239): 1, (14077, 0.540580987

Test model by comparing "similar" documents by hand <br>
<i>note: sourced from doc2vec documentation</i>

In [376]:
# Pick a random document from the test corpus and infer a vector from the model
doc_id = random.randint(0, len(test_corpus) - 1)
inferred_vector = model.infer_vector(test_corpus[doc_id])
print(inferred_vector)
sims = model.dv.most_similar([inferred_vector], topn=len(model.dv))

# Compare and print the most/median/least similar documents from the train corpus
print('Test Document ({}): «{}»\n'.format(doc_id, ' '.join(test_corpus[doc_id])))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_corpus[sims[index][0]].words)))

Test Document (3275): «can someone explain this sym craze»

SIMILAR/DISSIMILAR DOCS PER MODEL Doc2Vec(dm/m,d300,n5,w5,mc25,s0.001,t3):

MOST (124709, 0.3440297842025757): «what is the realistic potential of this sym craze»

MEDIAN (30619, 0.001687161042355001): «sym now playing on the just got inspired by wsb community to doodle this piece am just small retail investor and am holding sym until we all harvest those together upvote if you are holding sym buying more»

LEAST (94133, -0.26937946677207947): «why delete all the sym forums of reddit how weird»



If model is good, save it for future use

In [ ]:
pickle.dump(model, open('doc2vec_model.p', "wb"))

